In [4]:
pip install git+https://github.com/GJG1203/RobustConAcq.git

  Cloning https://github.com/GJG1203/RobustConAcq.git to c:\users\gertj\appdata\local\temp\pip-req-build-gccgib18
  Resolved https://github.com/GJG1203/RobustConAcq.git to commit 3768f3b1fedfe720d9dcbad66ce4f163caa6f246
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   -- ------------------------------------- 0.8/11.0 MB 5.6 MB/s eta 0:00:02
   --------- ------------------------------ 2.6/11.0 MB 6.6 MB/s eta 0:00:02
   -------------- ------------------------- 3.9/11.0 MB 6.3 MB/s eta 0:00:02
   ------------------ --------------------- 5.0/11.0 MB 6.3 MB/s eta 0:00:01
   ----------------------- ---------------- 6.6/11.0 MB 6.4 MB/s eta 0:0

  Running command git clone --filter=blob:none --quiet https://github.com/GJG1203/RobustConAcq.git 'C:\Users\gertj\AppData\Local\Temp\pip-req-build-gccgib18'


In [5]:
print('import succeeded')

import succeeded
